In [19]:
import os
import zipfile
import numpy as np
import pandas as pd
import matplotlib
import torch
import matplotlib.pyplot as plt
# from tqdm import tqdm_notebook
%matplotlib inline 
# from google.colab.patches import cv2_imshow
from IPython.display import HTML #imports to play videos
from base64 import b64encode 
import cv2 as cv
#from skimage.measure import compare_ssim
import glob
import time
from PIL import Image
from facenet_pytorch import MTCNN, InceptionResnetV1, extract_face
from tqdm import tqdm
from efficientnet_pytorch import EfficientNet
import math
import pickle
from functools import partial
from collections import defaultdict

from PIL import Image
from glob import glob

import cv2
import skimage.measure
import albumentations as A
from tqdm.notebook import tqdm 
#from albumentations.pytorch import ToTensor 
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torchvision.models.video import mc3_18, r2plus1d_18

from facenet_pytorch import MTCNN
from torchsummary import summary


import sys
import sklearn

import cv2
import pandas as pd
import numpy as np

import plotly.graph_objs as go
from plotly.offline import iplot
from matplotlib import pyplot as plt
from torchvision import models, transforms
from torch.utils.data import WeightedRandomSampler

import matplotlib.pyplot as plt

plt.rc('font', size=14)
plt.rc('axes', labelsize=14, titlesize=14)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)


device = 'cuda'

SOURCE_FOLDER = r"D:\dfdc"
TRAIN_FOLDER = os.path.join(SOURCE_FOLDER, 'dfdc_videos', "dfdc_train_part_1") 
TEST_FOLDER = os.path.join(SOURCE_FOLDER, 'dfdc_videos', "dfdc_train_part_0") 

TRAIN_FOLDER_CROPS = os.path.join(SOURCE_FOLDER, 'crops', "dfdc_train_part_1") 
TEST_FOLDER_CROPS = os.path.join(SOURCE_FOLDER, 'crops', "dfdc_train_part_0") 
BATCH_SIZE = 64
EPOCHS = 10

In [20]:
from glob import glob
import json
def get_videos_from_folder(folder_path):
    # List all files in the specified folder
    files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
    if 'metadata.json' in files:
        files.remove('metadata.json')
    full_paths = [os.path.join(folder_path, f) for f in files]
    return full_paths

def get_videos_basenames_from_folder(folder_path):
    # List all files in the specified folder
    files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
    if 'metadata.json' in files:
        files.remove('metadata.json')
    return files

def get_original_with_fakes(root_dir):
    pairs = []
    for json_path in glob(os.path.join(root_dir, "*/metadata.json")):
        with open(json_path, "r") as f:
            metadata = json.load(f)
        for k, v in metadata.items():
            original = v.get("original", None)
            if v["label"] == "FAKE":
                pairs.append((original[:-4], k[:-4] ))

    return pairs


metadata_val = pd.read_json(TEST_FOLDER+ '/metadata.json')
metadata_train = pd.read_json(TRAIN_FOLDER+ '/metadata.json')
#metadata['aaqaifqrwn.mp4']['label']

def sample_weights(metadata):
    video_fake_real_labels = metadata.iloc[0]
    class_counts = video_fake_real_labels.value_counts()
    class_weights = 1 / class_counts
    return [1/class_counts[i] for i in video_fake_real_labels] # for undersampling

train_weights_sampler = WeightedRandomSampler(weights=sample_weights(metadata_train), num_samples=len(metadata_train.iloc[0]), replacement=True)
val_weights_sampler =WeightedRandomSampler(weights=sample_weights(metadata_val), num_samples=len(metadata_val.iloc[0]), replacement=True)

In [21]:
# validation set will be 00
metadata_val = pd.read_json(TEST_FOLDER+ '/metadata.json')
metadata_train = pd.read_json(TRAIN_FOLDER+ '/metadata.json')
video_fake_real_labels = metadata_train.iloc[0]
real_df = video_fake_real_labels[video_fake_real_labels == 'REAL'].keys()
fake_df = video_fake_real_labels[video_fake_real_labels == 'REAL'].keys()

In [22]:
class DeepfakeDataset(Dataset):
    def __init__(self, metadata, device, mode='train'):
        self.device = device
        self.folder_path = TRAIN_FOLDER if mode=='train' else TEST_FOLDER
        self.folder_path_crops = TRAIN_FOLDER_CROPS if mode=='train' else TEST_FOLDER_CROPS
        self.video_fake_real_labels = metadata.iloc[0]

    def __len__(self):
        return len(self.video_fake_real_labels)

    def __getitem__(self, idx):
        filename = self.video_fake_real_labels.index[idx]
        label = 0 if self.video_fake_real_labels[filename] == 'REAL' else 1
        crops_folder_path = os.path.join(self.folder_path_crops, filename[:-4])
        crops_list = [os.path.join(crops_folder_path, f) for f in os.listdir(crops_folder_path) if os.path.isfile(os.path.join(crops_folder_path, f))]
        # get crops from folder
        crop_tensors = []
        
        for crop_file in crops_list:
            loaded_tensor = torch.load(crop_file)
            crop_tensors.append(loaded_tensor)
        
        crop_tensors = torch.stack(crop_tensors, dim=0) # stack tensors. this is a "batch"
        labels = torch.tensor([label for i in range(len(crops_list))]) # label for each frame is the same as video label

        return crop_tensors, labels

In [23]:
#pretrained_weights_path = 'noisy-student-efficientnet-b4.pth' # noisy student
#model = EfficientNet.from_name('efficientnet-b4')
#model = EfficientNet.from_name('efficientnet-b1')
#model.load_state_dict(torch.load(pretrained_weights_path, map_location=torch.device(device)))
#model._fc = nn.Linear(in_features=model._fc.in_features, out_features=1)
class EffnetTest(nn.Module):
    def __init__(self):
        super(EffnetTest, self).__init__()
        self.model = EfficientNet.from_pretrained(f"efficientnet-b0", num_classes=1)
        #self.model._norm_layer = nn.GroupNorm(num_groups=32, num_channels=3)
        #for i in range(4):
        #    for param in self.model._blocks[i].parameters():
        #        param.requires_grad = False

    def forward(self, x):
        x = self.model(x)
        return torch.sigmoid(x).squeeze()
model = EffnetTest()

Loaded pretrained weights for efficientnet-b0


In [24]:
summary(model, (3,224,224))

Layer (type:depth-idx)                                  Output Shape              Param #
├─EfficientNet: 1-1                                     [-1, 1]                   --
|    └─Conv2dStaticSamePadding: 2-1                     [-1, 32, 112, 112]        --
|    |    └─ZeroPad2d: 3-1                              [-1, 3, 225, 225]         --
|    └─BatchNorm2d: 2-2                                 [-1, 32, 112, 112]        64
|    └─MemoryEfficientSwish: 2-3                        [-1, 32, 112, 112]        --
|    └─ModuleList: 2                                    []                        --
|    |    └─MBConvBlock: 3-2                            [-1, 16, 112, 112]        1,448
|    |    └─MBConvBlock: 3-3                            [-1, 24, 56, 56]          6,004
|    |    └─MBConvBlock: 3-4                            [-1, 24, 56, 56]          10,710
|    |    └─MBConvBlock: 3-5                            [-1, 40, 28, 28]          15,350
|    |    └─MBConvBlock: 3-6                  

Layer (type:depth-idx)                                  Output Shape              Param #
├─EfficientNet: 1-1                                     [-1, 1]                   --
|    └─Conv2dStaticSamePadding: 2-1                     [-1, 32, 112, 112]        --
|    |    └─ZeroPad2d: 3-1                              [-1, 3, 225, 225]         --
|    └─BatchNorm2d: 2-2                                 [-1, 32, 112, 112]        64
|    └─MemoryEfficientSwish: 2-3                        [-1, 32, 112, 112]        --
|    └─ModuleList: 2                                    []                        --
|    |    └─MBConvBlock: 3-2                            [-1, 16, 112, 112]        1,448
|    |    └─MBConvBlock: 3-3                            [-1, 24, 56, 56]          6,004
|    |    └─MBConvBlock: 3-4                            [-1, 24, 56, 56]          10,710
|    |    └─MBConvBlock: 3-5                            [-1, 40, 28, 28]          15,350
|    |    └─MBConvBlock: 3-6                  

In [25]:
class EarlyStopper:
    def __init__(self, patience=5, verbose=True):
        self.patience = patience
        self.counter = 0
        self.best_score = float('inf')  # Initialize to positive infinity
        self.verbose = verbose

    def early_stop(self, val_loss, model):
        if val_loss < self.best_score:
            self.best_score = val_loss  # Update best validation loss
            self.counter = 0  # Reset patience counter
            torch.save(model.state_dict(), 'best_model.pth')  # Save the best model
        else:
            self.counter += 1  # Increment patience counter
           
        if self.counter >= self.patience:
            if self.verbose:
                print("Early stopping...")
            return True
        return False

In [26]:
train_dataset = DeepfakeDataset(metadata_train, device, mode='train')
train_loader = DataLoader(train_dataset, shuffle=False, sampler=train_weights_sampler, batch_size=1, num_workers=0) # process video 1 by 1
val_dataset = DeepfakeDataset(metadata_val, device, mode='val')
val_loader = DataLoader(val_dataset, shuffle=False, sampler=val_weights_sampler, batch_size=1, num_workers=0) # process video 1 by 1

# train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, num_=48, pin_memory=True)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-6)
loss_fn = nn.BCELoss()

train_loss_history = []
train_accuracy_history = []
val_loss_history = []
val_accuracy_history = []


In [29]:
epochs = 100
model = model.to(device)
earlystopper = EarlyStopper()
# mtcnn unable to detect gan/diffusion faces
#data = data.to(device)  # Move data to GPU if available
# half precision
# A6000 at desktop
for epoch in range(epochs): 
    running_loss = 0
    correct = 0
    total = 0
    model.train()
    for data, labels in train_loader:
        data = data.squeeze(0)
        labels = labels.squeeze(0)
        labels = labels.to(device).float()
        optimizer.zero_grad()  # clear previous gradients

        data = data.to(device).float()
        outputs = model(data)
        losses = loss_fn(outputs, labels)
        running_loss += losses.item()  # accumulate loss
        predicted = (outputs >= 0.5).float() # calculate if label is 0 or 1
        losses.backward()
        correct += (predicted == labels).sum().item() 
        total += labels.size(0)
        print(losses)

    average_train_loss = running_loss / total
    average_train_accuracy = correct / total
    train_loss_history.append(average_train_loss)
    train_accuracy_history.append(average_train_accuracy)
    

    model.eval()
    running_loss = 0
    correct = 0
    total = 0 
    for data, labels in val_loader:
        data = data.squeeze(0)
        labels = labels.squeeze(0)
        labels = labels.to(device).float()
        data = data.to(device).float()

        outputs = model(data)
        losses = loss_fn(outputs, labels)
        running_loss += losses.item()  # accumulate loss
        predicted = (outputs >= 0.5).float() # calculate if label is 0 or 1
        losses.backward()
        correct += (predicted == labels).sum().item() 
        total += labels.size(0)
    average_val_loss = running_loss / total
    average_val_accuracy = correct / total
    val_loss_history.append(average_val_loss)
    val_accuracy_history.append(average_val_accuracy)
    print(f"Epoch [{epoch}/{epochs}], Train Loss: {average_train_loss:.4f}, Train Accuracy: {average_train_accuracy:.4f}, Validation Loss: {average_val_loss:.4f}, Validation Accuracy: {average_val_accuracy:.4f}")
    #if earlystopper.early_stop(average_val_loss, model):
    #    break 
torch.save(model.state_dict(), 'model.pth')

C:\Users\Sheryl\AppData\Local\Temp\ipykernel_10328\3301423534.py:20: FutureWarning:

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.



tensor(0.7471, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.6435, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.6465, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.6432, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.6478, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.7528, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.7549, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.6444, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.6512, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.6341, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.7643, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.6518, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.7558, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.6503, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor

KeyboardInterrupt: 

In [11]:
torch.save(model.state_dict(), 'model.pth')

In [9]:
#pretrained_weights_path = 'noisy-student-efficientnet-b4.pth' # noisy student
#model = EfficientNet.from_name('efficientnet-b4')
#model.load_state_dict(torch.load(pretrained_weights_path, map_location=torch.device(device)))